## GLACIER AREA CALCULATOR ### 

This project will primarily use the USGS Python Package to identify, download and present Landsat imagery from 1985, 2005 and 2025 across several glacier mountain ranges across the world. 
The mountain ranges will include:
- Kenai Mountains (Alaska)
- Swiss Alps
- *Insert Greenland/Himalayas maybe?*
This script will download and present the images in False Colour Composite, then will generate a classification to identify the ice area on the images and this classification will be used in each image to calculate the change in glacier area across glaciated areas on a global scale.

##TO DO LIST FOR THE STUDY## 


----KENAI MOUNTAINS----
- Identify Landsat information on the USGS EE site and get the correct WRS path, etc.
- Download Landsat Images for Kenai Mountains first
- Clasification
- Area change calculation
- See if you can edit the image to show only the selected mountain range (Crop Stacked Landsat TIF)
- Maybe include some help() but ask Bob if it is something that is expected or required

----Swiss Alps-----
- Same as above
- Try to select area with Alestchgletscher

----GREENLAND/HIMALAYAS----
- Same as above


---AFTER LANDSAT DONE-----
- Generate a comparison Table/Template for all results acquired
- IF YOU HAVE TIME, maybe do a couple more glacier mountains
- Keep it that the script can be used to generate area as long as the correct info is input (ID, WRS, ETC..)
- Ask if an ArcGIS element is expected for the assignment
- Ask/Check how the how-to-guide is to be formatted (separate word doc in Git Repos/in the README/....
- Check how the final repos is to be submitted



In [ ]:
import usgs
import os
import requests

In [ ]:
from usgs import api

In [ ]:
# Open and read the Username and Api Key/Token for the USGS account
with open('C:/Users/couse/.usgs_user', 'r') as usertext:
    username = usertext.read()
with open('C:/Users/couse/.usgs_token', 'r') as tokentext:
    password = tokentext.read()
    
# Log in using the above information in the script attached below
login = api.login(username, password, save=True)

#The API Key (token) is extracted separately as it will be used in the script to search for and download Landsat imagery
api_key = login['data']

To identify and search for the required Landsat scenes in this study we are going to use the api.scene_search() function as this will be the primary method for getting the correct images before we download them. Below is some of the information we will need for the search

In [ ]:
help(api.scene_search)

For the scene searches in this study we are going to need the following:
- Dataset ID (dataset)
- Latitude and Longitude (lat and lng)
- Earth Explorer allows us to create a bounding box to get available images that cover those areas (ll and ur - Lower left corner and Upper Right corner for the bounding box respectively)
- Start and End date (start_date and end_date)
- Filters to get the correct images (where) (WRS Path, Cloud Cover, etc.)
- Max Results incase there are multiple available images (max_results)
- The extracted Api key (api_key)

Future adaptations could include filtering image selection based on cloud coverage to ensure that images used are cloud free.

We are going to use dataset_search to identify what Landsat datasets are available and importantly the exact 'dataset' which we will use to download and extract the specific scenes needed for this study

In [ ]:
# Landsat imagery comes from the EarthExplorer dataset (node), we will do a search and print out all Landsat datasets in the Earth Explorer node
satellite_avail = api.dataset_search("landsat", "EE")
for dataset in satellite_avail['data']:
        print(dataset['datasetAlias']) #Prints out names of all available datasets

All available scenes are printed above. In this project we are going to use the 'landsat_tm_c2_l1' (~1985 and ~2004) and the 'landsat_ot_c2_l1' (2024) datasets to acquire imagery of the selected glacier mountainscapes. The next step is to search for the Landsat images that will be used for the study. You can get the Bounding box coordinates and landsat image information on the USGS Earth Explorer website = https://earthexplorer.usgs.gov/. In this study, a set date has been used to get the exact image required, however the date range can be adjusted to get see the available products ether across a range of dates or on different days.

In [ ]:
help(api.scene_search)


In [ ]:
# In this cell we will use the search function to get the product id of the imager used in the study
results = api.scene_search(
    dataset = "landsat_tm_c2_l1", #Landsat dataset used for 1986 image
    lat = 60.09345, # The latitude of the image
    lng = -151.07395, # The longtitude of the image
    ll = {"longitude" : -151.7075, "latitude" : 59.1140}, # The Latitude and Longtitude of the Lower left corner of the Bounding Box in EE
    ur = {"longitude" : -150.1913, "latitude" : 59.9276}, # The Latitude and Longtitude of the Upper Right corner of the Bounding Box in EE
    start_date = "1986-09-11", # The start date of the search range
    end_date = "1986-09-13", # The end date of the search range
    api_key=api_key # The Users Api key
)
#The next lines of script will loop through each 'scene' that appears in the results and will print out the 'entity ID' which will be used in the dataset downlaod.
for scene in results['data']['results']:
    print("The Entity ID of the Landsat scene is",(scene['entityId'])) 

Now that the Entity ID is known we can now download the Band files which we will use to build a false colour composite image of the Kenai Mountains.

In [ ]:
#This cell will provide the different download options for the Landsat dataset
api.dataset_download_options("landsat_tm_c2_l1")                             

In this study we are going to use the 'Landsat Collection 2 Level-1 Product Bundle' to download all the different bands. The product ID code with the downlaod option will be used in the download request to download zip file into the repository.

In [ ]:
help(api.dataset_download_options)

In [ ]:
help(api.download_request)

In [ ]:
# ADD DOCSTRINGS FOR THIS CELL

os.makedirs("KenaiMountains1986", exist_ok=True) # Creates a new folder for the downloaded Landsat images folder

KM_1986 = api.download_request(
    dataset="landsat_tm_c2_l1",
    entity_id="LT50690181986255XXX05",
    product_id="5e83d0a0f94d7d8d",
)

 # Download the first file
if 'data' in KM_1986 and 'availableDownloads' in KM_1986['data']:
    KM_1986Avail = KM_1986['data']['availableDownloads']
    if KM_1986Avail:
        KM1986downloadurl = KM_1986Avail[0]['url']
        KM_1986File = os.path.join("KenaiMountains1986", f"LT50690181986255XXX05.tar")
        KM1986url = requests.get(KM1986downloadurl, stream=True)
        
        with open(KM_1986File, "wb") as f:
            for chunk in KM1986url.iter_content(chunk_size=8192):
                if chunk:
                    f.write(chunk)
                    
print("Landsat download complete!")

In [ ]:
help(requests.get)

In [ ]:
api.download_options(
    dataset="landsat_tm_c2_l1",
    entity_ids="LT50690181986255XXX05"
)


In [ ]:
# Logs out the existing USGS account for the script
api.logout()